# Introduction

The following is an analysis of data released by the Office of the Chief Judge for the Circuit Court of Cook County on May 9, 2019. It covers all defendants released on bail between July 1, 2016 and December 31, 2018 -- roughly 15 months before and 15 months after the introduction of Chief Judge Evans's bail reform order on September 18, 2017. The original data can be found here: http://www.cookcountycourt.org/HOME/BailReform.aspx. I've also uploaded each separate CSV file to the Google Drive folder for fact-checking.

## Overview

My original process of analysis was somewhat exploratory and iterative, so I'm creating this document as a clean version for fact-checking purposes. The first section shows how I cleaned the data and prepared it for analysis, and the sections that follow show how I calculated each statistic that appears in the article, roughly in the order they appear. 

## Checking My Work

If you'd like, you can replicate my analysis using the Jupyter Notebook file I've included in the Google Drive. To do that, you'll need to download Jupyter, if you don't have it already, which can be done by following the instructions here: https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html

You will also need the `numpy`, `pandas`, `matplotlib`, and `seaborn` Python packages, which can be installed using either `pip` or `conda`. 

If you're not familiar with Python, you can still follow along in this document to see how I calculated everything. I tried to include detailed explanations wherever possible, but if you have any questions, don't hesitate to reach out: ethan.corey@theappeal.org.

# Initial Setup and Data Cleaning

The data is stored in six Excel files, which I downloaded and converted to CSVs, all saved as `cook_bail_seg_X.csv`, where `X` is the segment number for the file. I've uploaded the CSV files to the Google Drive for fact-checking. The segments do not appear to be in any particular order, though it doesn't really matter, since I combine them into a single `pandas.DataFrame` below.

I then used Python's `pandas` data analysis library to clean and analyze the data.

In [50]:
from typing import Any, Mapping, Callable, List, Union, Sequence, Type, TypeVar
import re

import pandas as pd
import scipy.stats as scs
import numpy as np


# Configuration for data visualization packages
plt.rc("font", size=14)
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)


# Each csv is loaded as an individual data frame
seg1 = pd.read_csv('cook_bail_seg_1.csv')
seg2 = pd.read_csv('cook_bail_seg_2.csv')
seg3 = pd.read_csv('cook_bail_seg_3.csv')
seg4 = pd.read_csv('cook_bail_seg_4.csv')
seg5 = pd.read_csv('cook_bail_seg_5.csv')
seg6 = pd.read_csv('cook_bail_seg_6.csv')

# A combined data frame is created by concatenating the six individual data frames
combined = pd.concat([seg1, seg2, seg3, seg4, seg5, seg6])

### Initial Data Format

Each row contains 23 columns, containing the following information:

| Field Name          | Field Description                                    | Field Format | NumValues            |
|---------------------|------------------------------------------------------|--------------|----------------------|
| RecordID            | Unique ID                                            | `str`        | 58,979 (unique)      |
| PrePostGOInidcator  | Flag indicating whether record is pre or post reform | `str`        | 2                    |
| ChargeCategory      | Charge Category                                      | `str`        | 6                    |
| ChargeClass         | Charge Severity Level (all felonies)                 | `str`        | 7                    |
| AccompanyingMatter  | Indicator of probation/parole or bond violation      | `str`        | 2                    |
| Gender              | Gender                                               | `str`        | 2                    |
| RaceEthnicity       | Race or Ethnicity                                    | `str`        | 4                    |
| AgeCategory         | Age Range                                            | `str`        | 4                    |
| PSASuperRec         | Public Safety Assessment Supervision Recommendation  | `str`        | 6                    |
| FTAScale            | Public Safety Assessment FTA Risk Score              | `str`        | 3                    |
| NCAScale            | Public Safety Assessment New Criminal Activity Score | `str`        | 3                    |
| PSAVioFlag          | Public Safety Assessment Violence Flag               | `str`        | 2                    |
| InitialBondOrder    | Initial Bond Set In Case                             | `str`        | 5                    |
| IBondOrder          | I-Bond (ROR) Flag                                    | `str`        | 2                    |
| DBondOrder          | D-Bond (Deposit Bond) Flag                           | `str`        | 2                    |
| EMOrder             | Electronic Monitoring Flag                           | `str`        | 2                    |
| NoBailOrder         | No Bail Flag                                         | `str`        | 2                    |
| DBond10pAmtCategory | 10 percent Bond Deposit Amount Range                 | `str`        | 6                    |
| DBond10pAmt         | 10 percent Bond Deposit Amount                       | `float`      | 118                  |
| PretrialRelease     | Pretrial Release Flag                                | `str`        | 2                    |
| FailToAppear        | FTA Flag                                             | `str`        | 3                    |
| NewCrimActivity     | New Criminal Activity Flag                           | `str`        | 3                    |
| NewVioCrimActivity  | New Violent Crime Flag                               | `str`        | 2                    |

Definitions for most of the data fields can be found in the report that Cook County Circuit Court released alongside the bond data: http://www.cookcountycourt.org/Portals/0/Statistics/Bail%20Reform/Bail%20Reform%20Report%20FINAL%20-%20%20Published%2005.9.19.pdf

It's also saved in the `Cook_County` folder in the Google Drive.

An example record contains the following information:

In [51]:
combined.iloc[0]

RecordID                            1.00
PrePostGOInidcator        Pre G.O. 18.8A
ChargeCategory                  Property
ChargeClass                      Class 3
AccompanyingMatter        No VOP or VOBB
Gender                            Female
RaceEthnicity                      Black
AgeCategory                        36-45
PSASuperRec                  If Rel, PTM
FTAScale                          1 or 2
NCAScale                          1 or 2
PSAVioFlag              No Violence Flag
InitialBondOrder                    COEM
IBondOrder                    Other Bond
DBondOrder                    Other Bond
EMOrder                               EM
NoBailOrder                   Other Bond
DBond10pAmtCategory                  NaN
DBond10pAmt                          NaN
PretrialRelease        Pre-Trial Release
FailToAppear            Release - No FTA
NewCrimActivity         Release - No NCA
NewVioCrimActivity            No Vio Off
Name: 0, dtype: object

### Cleaning the Data

To ease my analysis, I'm making the following transformations on the data. 

| Field Name          | Current Data Format  | Transformation                          | NaN Filling                     |
|---------------------|----------------------|-----------------------------------------|---------------------------------|
| RecordID            | `str` (unique)       | Convert to `int`                        | N/A                             | 
| PrePostGOInidcator  | `str` (flag)         | Convert to `bool` (also fix field name) | N/A                             | 
| ChargeCategory      | `str` (categorical)  | Convert to `int`                        | N/A                             | 
| ChargeClass         | `str` (categorical)  | Convert to `int`                        | N/A                             | 
| AccompanyingMatter  | `str` (flag)         | Convert to `bool`                       | N/A                             | 
| Gender              | `str` (categorical)  | Convert to `int`                        | N/A                             | 
| RaceEthnicity       | `str` (categorical)  | Convert to `int`                        | N/A                             | 
| AgeCategory         | `str` (categorical)  | Convert to `int`                        | N/A                             | 
| PSASuperRec         | `str` (categorical)  | Convert to `int`                        | N/A                             | 
| FTAScale            | `str` (categorical)  | Convert to `int`                        | N/A                             | 
| NCAScale            | `str` (categorical)  | Convert to `int`                        | N/A                             | 
| PSAVioFlag          | `str` (flag)         | Convert to `bool`                       | N/A                             | 
| InitialBondOrder    | `str` (flag)         | Convert to `bool`                       | N/A                             | 
| IBondOrder          | `str` (flag)         | Convert to `bool`                       | N/A                             | 
| DBondOrder          | `str` (flag)         | Convert to `bool`                       | N/A                             | 
| EMOrder             | `str` (flag)         | Convert to `bool`                       | N/A                             | 
| NoBailOrder         | `str` (flag)         | Convert to `bool`                       | N/A                             | 
| DBond10pAmtCategory | `str` (categorical)  | Convert to `int`                        | Stet (filter out when analyzing |
| DBond10pAmt         | `float` (continuous) | Stet                                    | Stet (filter out when analyzing |
| PretrialRelease     | `str` (flag)         | Convert to `bool`                       | N/A                             | 
| FailToAppear        | `str` (categorical)  | Convert to `int`                        | N/A                             | 
| NewCrimActivity     | `str` (categorical)  | Convert to `int`                        | N/A                             | 
| NewVioCrimActivity  | `str` (flag)         | Convert to categorical as `int`         | N/A                             | 

#### Helper Functions

Functions I'm defining to make the transformations to the data listed above. The doc-string for each function describes how it makes the transformation. I also use inline comments to make it clear how values are converted. I also use type hints to specify the input and output of each function.

##### Template Functions

These functions define how each general category of data conversion gets handled. Doc-strings explain the general mechanics of each category of transformation.

In [52]:
def map_cell(cell_map: Mapping[str, Any]) -> Callable[[str], Any]:
    """Template function for correcting cell data. Returns specified mapping, or None if no mapping specified."""
    def map_func(cell: str) -> Any:
        if cell in cell_map.keys():
            return cell_map[cell]
        else:
            return None
    return map_func

def map_cat_cell(cat_list: Sequence[str]) -> int:
    """Template function for mapping categorical cell data. Returns zero-indexed mapping """
    mapping = dict((j, i) for i, j in enumerate(cat_list))
    return map_cell(mapping)

def map_bool_cell(bool_map: Mapping[str, bool]) -> bool:
    """Template function for mapping boolean cell data. Returns boolean result of bool_map."""
    return map_cell(bool_map)

##### Individual Transformations

These functions are defined for each column transformation I'm making to the dataset. Most are constructed using one of the template functions defined above. Each function defines a specific mapping that will be applied to each cell in the column. Refer to doc-strings and inline comments if the mechanics are unclear.

In [53]:
def fix_id(id_: str) -> int:
    """Converts RecordID to int datatype"""
    no_comma = id_.replace(',', '')
    no_dec = re.sub(r'\.0*', '', no_comma)
    return int(no_dec)

# Replaces PrePostGOInidcator string with boolean using map_bool_cell.
fix_GO_flag = map_bool_cell(
    {
        'Post G.O. 18.8A': True,  # Post-reform
        'Pre G.O. 18.8A': False,  # Pre-reform
    }
)

# Converts ChargeCategory to numerical value using map_cat_cell, indexed in the order specified below.
fix_charge_cat = map_cat_cell(
    [
        'Drug',
        'Property',
        'Weapon',
        'Other',
        'Person-Violent',
        'Person',
    ]
)

# Converts ChargeClass to numerical value using map_cat_cell, indexed in the order specified below.
fix_charge_class = map_cat_cell(
    [
        'Class M',  # Misdemeanor
        'Class 4',  # Class 4 Felony (1-3 year minimum)
        'Class 3',  # Class 3 Felony (2-5 year minimum)
        'Class 2',  # Class 2 Felony (3-7 year minimum)
        'Class 1',  # Class 1 Felony (4-15 year minimum; 4-20 if second-degree murder)
        'Class X',  # Class X Felony (4-20 year minimum; life for first-degree murder)
        'Felony - Class Missing',
    ]
)

# Replaces AccompanyingMatter string with boolean using map_bool_cell.
fix_accompanying_matter = map_bool_cell(
    {
        'VOP or VOBB': True,     #  Violation of probation or bail bond 
        'No VOP or VOBB': False  # No violation of probation or bail bond
    }
)

# Converts Gender to numerical value using map_cat_cell, indexed in the order specified below.
fix_gender = map_cat_cell(
    [
        'Male',
        'Female',
    ]
)

# Converts RaceEthnicity to numerical value using map_cat_cell, indexed in the order specific below.
fix_race = map_cat_cell(
    [
        'Black',
        'White-Hispanic',
        'White Non-Hispanic',
        'Other',
    ]
)

# Converts AgeCategory to numerical value using map_cat_cell, indexed in the order specific below.
fix_age = map_cat_cell(
    [
        '25 and younger',
        '26-35',
        '36-45',
        '46 and older',
    ]
)

# Converts PSASuperRec to numerical value using map_cat_cell, indexed in the order specific below.
fix_PSASuperRec = map_cat_cell(
    [
        'RNC',              # Release no conditions
        'If Rel, PTM',      # If released, Pretrial Supervision (court date reminders & checks before each court date)
        'If Rel, PTS 1-3',  # If released, Pretrial Supervision & Conditions Monitoring & Monthly to Weekly face-to-face contact
        'If Rel, PTS HC',   # If released, PTS conditions plus curfew
        'If Rel, COEM',     # If released, electronic monitoring
        'If Rel, MC',       # Any combination of supervision and conditions (that do not include I-Bond)
    ]
)


# Converts FTAScale or NCAScale to numerical value using map_cat_cell, indexed in the order specific below.
fix_FTAScale_NCAScale = map_cat_cell(
    [
        '1 or 2',  # Lowest risk
        '3 or 4',  # Moderate risk
        '5 or 6',  # Highest risk
    ]
)

# Converts PSAVioFlag string to boolean using map_bool_cell.
fix_VioFlag = map_bool_cell(
    {
        'Violence Flag': True,
        'No Violence Flag': False,
    }
)


# Converts InitialBondOrder to numerical value using map_cat_cell, indexed in the order specific below.
fix_InitBond = map_cat_cell(
    [
        'I-Bond',   # ROR
        'D-Bond',   # Deposit Bond
        'C-Bond',   # Cash Bond
        'COEM',     # Electronic Monitoring
        'No Bail',  # No bail
    ]
)

# Converts IBondOrder string to boolean using map_bool_cell.
fix_IBond = map_bool_cell(
    {
        'I-Bond': True,       # RORed
        'Other Bond': False,  # No ROR
    }
)

# Converts DBondOrder string to boolean using map_bool_cell.
fix_DBond = map_bool_cell(
    {
        'D/C-Bond': True,     # Money bond set
        'Other Bond': False,  # No money bond set
    }
)

# Converts EMOrder string to boolean using map_bool_cell.
fix_EM = map_bool_cell(
    {
        'EM': True,           # Electronic monitoring ordered
        'Other Bond': False,  # No electronic monitoring ordered
    }
)

# Converts NoBailOrder string to boolean using map_bool_cell.
fix_NoBail = map_bool_cell(
    {
        'No Bail': True,      # Held without bail
        'Other Bond': False,  # Some bail set or ROR
    }
)


# Converts DBond10pAmtCategory to numerical value using map_cat_cell, indexed in the order specific below.
fix_DBondCat = map_cat_cell(
    [
        '$1-$500',
        '$501-$1,000',
        '$1,001-$3,000',
        '$3,001-$5,000',
        '$5,001+',
        'Missing',
        'No Bond Set',
    ]
)

# Converts PretrialRelease string to boolean using map_bool_cell.
fix_PretrialRelease = map_bool_cell(
    {
        'Pre-Trial Release': True,
        'No Pre-Trial Release': False,
    }
)

# Converts FailToAppear to numerical value using map_cat_cell, indexed in the order specific below.
fix_ReleaseFTA = map_cat_cell(
    [
        'Release - No FTA',
        'Release FTA',
        'No Release',
    ]
)

# Converts NewCrimActivity to numerical value using map_cat_cell, indexed in the order specific below.
fix_ReleaseNCA = map_cat_cell(
    [
        'Release - No NCA',  # Released without new arrest
        'Release - NCA',     # Released with new arrest
        'No Release',        # No release
    ]
)

def fix_NewVio(row: str) -> int:
    """Convert NewVioCrimActivity to numerical value, as specified below. This transformation is 
    performed using pd.DataFrame.apply, since it requires access to multiple columns.
    """
    pt_release = row['PretrialRelease']
    new_vio = row['NewVioCrimActivity']
    
    if pt_release == 'Pre-Trial Release':
        if new_vio == "No Vio Off":                                  # Released without arrest for violent offense
            return 0
        elif new_vio == "Vio Off":                                   # Released with new arrest for violent offense
            return 1
        else:
            return None
    else:                                                            # Not released
        return 2

#### Applying the Transformations

Now, each transformation defined above is applied to its specified column, and the resulting `pandas.Series` objects are added to a new `pandas.DataFrame`. For future convenience, I also fix the spelling error in the `PrePostGOInidcator` column that appears in the original dataset.

In [54]:
transformed = pd.DataFrame()

transformed['RecordID'] = combined['RecordID'].map(fix_id)
transformed['PrePostGOIndicator'] = combined['PrePostGOInidcator'].map(fix_GO_flag)
transformed['ChargeCategory'] = combined['ChargeCategory'].map(fix_charge_cat)
transformed['ChargeClass'] = combined['ChargeClass'].map(fix_charge_class)
transformed['AccompanyingMatter'] = combined['AccompanyingMatter'].map(fix_accompanying_matter)
transformed['Gender'] = combined['Gender'].map(fix_gender)
transformed['RaceEthnicity'] = combined['RaceEthnicity'].map(fix_race)
transformed['AgeCategory'] = combined['AgeCategory'].map(fix_age)
transformed['PSASuperRec'] = combined['PSASuperRec'].map(fix_PSASuperRec)
transformed['FTAScale'] = combined['FTAScale'].map(fix_FTAScale_NCAScale)
transformed['NCAScale'] = combined['NCAScale'].map(fix_FTAScale_NCAScale)
transformed['PSAVioFlag'] = combined['PSAVioFlag'].map(fix_VioFlag)
transformed['InitialBondOrder'] = combined['InitialBondOrder'].map(fix_InitBond)
transformed['IBondOrder'] = combined['IBondOrder'].map(fix_IBond)
transformed['DBondOrder'] = combined['DBondOrder'].map(fix_DBond)
transformed['EMOrder'] = combined['EMOrder'].map(fix_EM)
transformed['NoBailOrder'] = combined['NoBailOrder'].map(fix_NoBail)
transformed['DBond10pAmtCategory'] = combined['DBond10pAmtCategory'].map(fix_DBondCat)
transformed['DBond10pAmt'] = combined['DBond10pAmt']
transformed['PretrialRelease'] = combined['PretrialRelease'].map(fix_PretrialRelease)
transformed['FailToAppear'] = combined['FailToAppear'].map(fix_ReleaseFTA)
transformed['NewCrimActivity'] = combined['NewCrimActivity'].map(fix_ReleaseNCA)
transformed['NewVioCrimActivity'] = combined.apply(fix_NewVio, axis=1)

An example of how a `transformed` record differs from the original `combined` record:

In [55]:
pd.DataFrame(
    {
        'combined' : combined.iloc[0],
        'transformed': transformed.iloc[0],
    }
)

,combined,transformed
AccompanyingMatter,No VOP or VOBB,False
AgeCategory,36-45,2
ChargeCategory,Property,1
ChargeClass,Class 3,2
DBond10pAmt,NaN,NaN
DBond10pAmtCategory,NaN,NaN
DBondOrder,Other Bond,False
EMOrder,EM,True
FTAScale,1 or 2,0
FailToAppear,Release - No FTA,0


# Release Outcomes by PSA Scores

The first part of my analysis compares how defendants fared upon release based on their scores on each of the PSA's three subscales: Failure to Appear (FTA), New Criminal Activity (NCA), and New Violent Criminal Activity (NVCA). Note that while the PSA does measure NVCA on a 1-6 scale like the other two sub-scales, it converts the score to a binary flag, with scores between 4 and 6 generating a "violence flag," and scores between 1 and 3 generating no flag. See https://drive.google.com/open?id=0B7dGJnakjIucdEhod2FHWkVVdC1uVzFtWFBFMEx5MXNGcWx3. 

The Cook County bond data does not include the exact score for any of the sub-scales. Rather, it uses the buckets 1-2, 3-4, and 5-6 for the FTA and NCA scales (corresponding to low, medium, and high risk), and it just records the presence or absence of a flag for the NVCA scale. 

In my analysis below, I compare defendants based on the categories present in the Cook County bond data. For FTA and NCA, I compare low-, medium-, and high-risk defendants. For NVCA, I compare defendants with the NVCA flag to those without the flag. For each scale, I conduct two outcomes comparisons. The first comparison looks at the outcomes in bond court itself: the release conditions defendants received, the average bond amount (if set), and the number/percentage of defendants released. The second comparison examines what happened to defendants who were released pretrial: whether they had an FTA, new criminal charge, or new violent criminal charge prior to case disposition. 

Note that there are a few confounding factors that could affect this second analysis. First, defendants with different risk scores were released at different rates, meaning that judges could be detaining the truly "dangerous" or "risky" defendants, thereby depressing the actual FTA/NCA/NVCA rates upon release. Second, when defendants received D-Bonds, they often spent some time behind bars before they were released, meaning that they likely spent less time on average on release than defendants who were released immediately on I-Bonds, which could lead to lower FTA/NCA/NVCA rates. Unfortunately, the Cook County data does not appear to give us a good way to tell how much impact this factor may have had on my results.

Ultimately, my analysis rests on the assumption that even if you account for these potential confounding factors, the differences in release outcomes for defendants with differing risk scores are relatively minor. Most experts I spoke with noted that these confounding factors existed, but said they thought the general finding was probably nonetheless accurate. You can also see how DeMichele et al. accounted for these similar problems in their analysis of the PSA in Kentucky: https://drive.google.com/open?id=1edGsSTY1F2GGOKd2dwV-4Xm2RTghTk0H.

## Helper Classes and Functions

The classes and functions defined below facilitate my analysis by allowing me to easily compare outcomes based on any variable I like. The lynchpin is the `Outcome_Table` class, which serves as the template for the subclasses I use to actually compare the outcomes. It defines a set of general methods useful for analyzing any combination of input variables and outcome measures, as well as two constructor methods allowing for the on-the-fly creation of new subclasses of `Outcome_Table`.

In [56]:
def count_selection(df: pd.DataFrame, sel: pd.Series) -> int:
    """Returns the number of rows in df that correspond to the boolean vector passed as sel.
    For instance, count_selection(transformed, transformed['IBondOrder']) would return 
    the number of defendants who received an I-Bond.
    """
    return len(df.loc[sel])
    
def count_released(df: pd.DataFrame) -> int:
    """Returns the number of defendants released pretrial."""
    return count_selection(df, df['PretrialRelease'])

def count_incarcerated(df: pd.DataFrame) -> int:
    """Returns the number of defendants incarcerated pretrial."""
    return count_selection(df, ~df['PretrialRelease'])

def count_IBond(df: pd.DataFrame) -> int:
    """Returns the number of defendants who received an I-Bond."""
    return count_selection(df, df['IBondOrder'])

def count_DBond(df: pd.DataFrame) -> int:
    """Returns the number of defendants who received an D-Bond."""
    return count_selection(df, df['DBondOrder'])

def count_EM(df: pd.DataFrame) -> int:
    """Returns the number of defendants who received court-ordered electronic monitoring."""
    return count_selection(df, df['EMOrder'])

def count_NoBail(df: pd.DataFrame) -> int:
    """Returns the number of defendants denied bail at their initial hearing. Note that a small 
    percentage of these defendants were able to successfully appeal their initial bail order and
    gain release. 
    """
    return count_selection(df, df['NoBailOrder'])

def count_FTA(df: pd.DataFrame) -> int:
    """Returns the number of defendants who receieved an FTA after being released."""
    return count_selection(df, df['FailToAppear'] == 1)

def count_NCA(df: pd.DataFrame) -> int:
    """Returns the number of defendants who receieved an NCA after being released."""
    return count_selection(df, df['NewCrimActivity'] == 1)

def count_NVCA(df: pd.DataFrame) -> int:
    """Returns the number of defendants who receieved an NVCA after being released."""
    return count_selection(df, df['NewVioCrimActivity'] == 1)

def count_any_failure(df: pd.DataFrame) -> int:
    """Returns the number of defendants who received an FTA, NCA, or NVCA after being released."""
    return len(df.loc[
        (df['FailToAppear'] == 1)
       |(df['NewCrimActivity'] == 1)
       |(df['NewVioCrimActivity'] == 1)
    ])

def average_DBond(df: pd.DataFrame) -> Union[int, float]:
    """Returns the average D-Bond, if any, defendants received at their bond hearing. Note that
    defendants are only required to pay 10 percent of the bond in order to secure their release.
    """
    dbonds = df.loc[pd.notna(df['DBond10pAmt'])]
    if len(dbonds):
        return 10 * int(dbonds['DBond10pAmt'].mean())
    else:
        return np.nan


class Outcome_Table(pd.DataFrame):
    """Base class for outcome tables defined below. Subclasses pd.DataFrame and defines general 
    methods used in each class.
    """
    def __init__(self, df: pd.DataFrame, groups: Sequence[str]) -> None:
        """Initializes an Outcome_Table instance. df is the dataframe used to calculate the outcome 
        table, and groups is a sequence of column names you wish to group the dataframe by. 
        """
        outcome_groups = df.groupby(groups)
        rows = [
            self.create_outcome_data(group[1])
            for group in outcome_groups
        ]
        rows.append(self.create_outcome_data(df))
        super().__init__(rows)
        self['Total'] = self.calc_totals(outcome_groups)
        labels = self.create_labels()
        labels.append('All Defendants')
        self.index = pd.Index(labels)
        
    def create_outcome_data(self, df: pd.DataFrame) -> pd.Series:
        """Generic method for creating outcome data. Defined separately in each 
        sub-class.
        """
        pass
    
    def calc_totals(self, groups: pd.core.groupby.generic.DataFrameGroupBy) -> pd.Series:
        """Calculates 'Total' column by finding the number of entries in each group."""
        group_totals = [len(group[1]) for group in groups]
        group_totals.append(sum(group_totals))
        return pd.Series(group_totals)
    
    def create_labels(self):
        """Generic method for creating outcome data. Defined separately in each 
        sub-class. The number of labels this function returns must equal the number of
        unique values for the column or combination of columns for which you wish to
        measure outcomes. For instance, an outcome table for the FTA subscale would
        define labels '1-2', '3-4', and '5-6' -- I do just that below.
        """
        pass
        
    def percentages(self, columns: Sequence[str]) -> pd.DataFrame:
        """Abstract method for transforming outcome table into a table of percentages.
        The columns argument specifies which columns to transform. 
        """
        new = self.apply(lambda x: round(100 * x / x['Total'], 3), axis=1).loc[:, columns]
        
        # Multiply by 200 because addition of 'Total' row causes self['Total'].sum() to double
        new['Total'] = self['Total'].apply(lambda x: round(200 * x / self['Total'].sum(), 3))
        return new
    
    def graph(self, *args, **kwargs):
        """Abstract method for creating graphs from outcome tables.
        Defined separately in each subclass.
        """
        pass
    
    @classmethod
    def define_cats(cls, name: str, labels: Sequence[str]) -> Type:
        """Generic method for creating Outcome_Table subclasses on the fly; allows measuring
        of custom categories. Inherits from whichever subclass of Outcome_Table calls this method.
        
        For instance, to create the NVCA_Outcome_Table class defined below, one could use this
        method as follows: 
        
        NVCA_Outcome_Table = Outcome_Table.define_cats(
            'NVCA_Outcome_Table', 
            ['No NVCA Flag', 'NVCA Flag'],
        )
        """
        return type(name, (cls,), {'create_labels': lambda self: labels})
    
    @classmethod
    def define_outcomes(cls, 
                        name: str, 
                        outcomes: Mapping[str, Callable[[pd.DataFrame], int]], 
                        percentages: Sequence[str]) -> Type:
        """Generic method for creating Outcome_Table subclasses on the fly; allows measuring
        of custom outcomes. Inherits from whichever subclass of Outcome_Table calls this method.
        
        For instance, to create the Hearing_Outcome_Table class defined below, one could use this
        method as follows: 
        
        Hearing_Outcome_Table = Outcome_Table.define_outcomes(
            'Hearing_Outcome_Table', 
            {
                "Incarcerated Pretrial": count_incarcerated,
                "Released Pretrial": count_released,
                "I-Bonds": count_IBond,
                "D-Bonds": count_DBond,
                "Average D-Bond (if set)": average_DBond,
                "Electronic Monitoring": count_EM,
                "Held Without Bond": count_NoBail,
            },
            [
                "Incarcerated Pretrial",
                "Released Pretrial",
                "I-Bonds",
                "D-Bonds",
                "Electronic Monitoring",
                "Held Without Bond",
            ],
        )
        """
        def init_func(self, df: pd.DataFrame, groups) -> None:
            super.__init__(df, groups)
            self.columns = list(outcomes.keys())
            
        return type(
            name, 
            (cls,), 
            {
                '__init__':  init_func,
                'create_outcome_data': lambda self, df: pd.Series(outcome(df) for outcome in outcomes),
                'percentages': lambda self: super.percentages(*percentages),
            }
        )

class Charge_Outcome_Table(Outcome_Table):
    """Base class for outcome tables that group data using
    'ChargeCategory'.
    """
    
    def create_labels(self):
        return [
            'Drug',
            'Property',
            'Weapon',
            'Other',
            'Person-Violent',
            'Person',
        ]
    
class FTA_NCA_Outcome_Table(Outcome_Table):
    """Base class for outcomes that group data using the
    three possible labels for the FTA and NCA scales.
    """
    
    def create_labels(self):
        """Creates label for each risk score group."""
        return [
            '1-2',
            '3-4',
            '5-6',
        ]
    
    
    
class NVCA_Outcome_Table(Outcome_Table):
    """Base class for outcomes that group data using the
    two possible labels for the NVCA flag.
    """
    
    def create_labels(self):
        """Creates label for each risk score group."""
        return [
            'No NVCA Flag',
            'NVCA Flag',
        ]
    
class DMF_Outcome_Table(Outcome_Table):
    """Base class for outcome tables that group data using 'PSASuperRec'."""
    
    def create_labels(self):
        return [
        'Release No Conditions',              
        'Pretrial Monitoring', 
        'If Released, Pretrial Superivision 1-3',  
        'If Released, PTS Plus Curfew',   
        'If Released, Court-Ordered Electronic Monitoring',     
        'If Released, Max Conditions',       
        ]

class Hearing_Outcome_Table(Outcome_Table):
    """Base class for creating hearing outcome table.
    Measures the number of defendants incarcerated and
    released pretrial, as well of the number of defendants
    who received each possible bond hearing outcome and the 
    total number of defendants.
    """
    
    def __init__(self, df: pd.DataFrame, groups: Sequence[str]) -> None:
        super().__init__(df, groups)
        self.columns = [
            "Incarcerated Pretrial",
            "Released Pretrial",
            "I-Bonds",
            "D-Bonds",
            "Average D-Bond (if set)",
            "Electronic Monitoring",
            "Held Without Bond",
            "Total",
        ]
    
    def create_outcome_data(self, df: pd.DataFrame) -> pd.Series:
        return pd.Series([
            count_incarcerated(df),
            count_released(df),
            count_IBond(df),
            count_DBond(df),
            average_DBond(df),
            count_EM(df),
            count_NoBail(df),
        ])
            
    def percentages(self) -> pd.DataFrame:
        return super().percentages([
            "Incarcerated Pretrial",
            "Released Pretrial",
            "I-Bonds",
            "D-Bonds",
            "Electronic Monitoring",
            "Held Without Bond",
        ])
    
class FTA_NCA_Hearing_Outcome_Table(Hearing_Outcome_Table, FTA_NCA_Outcome_Table):
    """Class for creating general FTA or NCA hearing outcome tables."""
    pass

class NVCA_Hearing_Outcome_Table(Hearing_Outcome_Table, NVCA_Outcome_Table):
    """Class for creating general FTA or NVCA hearing outcome tables."""
    pass

class DMF_Hearing_Outcome_Table(DMF_Outcome_Table, Hearing_Outcome_Table):
    """Class for DMF hearing outcome tables."""
    pass

class Release_Outcome_Table(Outcome_Table):
    """Base class for creating release outcome table. 
    Measures the number of FTAs, NCAs, and NVCAs for the 
    released defendants, as well as the total number of 
    released defendants.
    """
    
    def __init__(self, df: pd.DataFrame, groups: Sequence[str]) -> None:
        super().__init__(df, groups)
        self.columns = [
            "Released, FTAs",
            "Released, NCAs",
            "Released, NVCAs", 
            "Released, Any Failure",
            "Total",
        ]
    
    def create_outcome_data(self, df: pd.DataFrame) -> pd.Series:
        return pd.Series([
            count_FTA(df),
            count_NCA(df),
            count_NVCA(df),
            count_any_failure(df),
        ])
    
    def percentages(self) -> pd.DataFrame:
        return super().percentages([
            "Released, FTAs",
            "Released, NCAs",
            "Released, NVCAs",
            "Released, Any Failure"
        ])

class FTA_NCA_Release_Outcome_Table(Release_Outcome_Table, FTA_NCA_Outcome_Table):
    """Class for creating general FTA or NCA release outcome tables."""
    pass

class NVCA_Release_Outcome_Table(Release_Outcome_Table, NVCA_Outcome_Table):
    """Class for creating general NVCA release outcome tables."""
    pass


class DMF_Release_Outcome_Table(DMF_Outcome_Table, Release_Outcome_Table):
    """Class for DMF release outcome tables"""
    pass

## One More Data Transformation

Because release and detention rates changed pretty starkly after G.O. 18.8A, I'm also dividing my analysis into pre- and post-G.O. 18.8A periods. You'll notice that while the release rates change noticeably, the release outcomes (i.e., FTA rate, NCA rate, and NVCA rate) don't change by very much. When I cite figures in the article, I am citing post-G.O. 18.8A numbers unless otherwise noted.

In [57]:
pre_reform = transformed.loc[~transformed['PrePostGOIndicator']]
post_reform = transformed.loc[transformed['PrePostGOIndicator']]

pre_reform_released = pre_reform.loc[pre_reform['PretrialRelease']]
post_reform_released = post_reform.loc[post_reform['PretrialRelease']]

## Failure to Appear Subscale

This section examines differences in hearing and release outcomes based on each person's FTA subscale score. As mentioned above, I break the analysis into pre- and post-G.O. 18.8A periods, but the release outcomes across the two periods are more or less the same. The biggest difference is that release rates jumped substantially after the reforms, from about 72 percent to 81 percent. The increase was similar across risk scores. (Though also note that the percentage of defendants held *without* bond increased pretty dramatically as well, especially for those with the highest FTA risk scores.)

The key finding for my article is that defendants who received a five or a six on the FTA subscale (i.e., were "high risk" for getting an FTA) nonetheless made it to all court appearances roughly **71 percent** of the time, compared with **83 percent** of defendants overall. In other words, while the "high risk" defendants were significantly more likely to miss court appearances, the vast majority still made it to every hearing without a problem. 

### Hearing Outcomes

##### Post-Reform

In [58]:
post_reform_fta_hearing_outcomes = FTA_NCA_Hearing_Outcome_Table(post_reform, ['FTAScale'])
post_reform_fta_hearing_outcomes

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Average D-Bond (if set),Electronic Monitoring,Held Without Bond,Total
1-2,1645,11335,7586,3766,31150,786,837,12980
3-4,3289,11816,7592,4798,37210,1636,1077,15105
5-6,994,1353,580,920,44380,568,278,2347
All Defendants,5928,24504,15758,9484,35500,2990,2192,30432


##### Pre-Reform

In [59]:
pre_reform_fta_hearing_outcomes = FTA_NCA_Hearing_Outcome_Table(pre_reform, ['FTAScale'])
pre_reform_fta_hearing_outcomes

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Average D-Bond (if set),Electronic Monitoring,Held Without Bond,Total
1-2,2653,10194,4545,5693,113660,2393,116,12847
3-4,4579,9335,2751,7021,96250,3955,132,13914
5-6,880,906,203,1123,89460,435,19,1786
All Defendants,8112,20435,7499,13837,102860,6783,267,28547


#### Percentages

##### Post-Reform

In [60]:
post_reform_fta_hearing_outcomes.percentages()

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Electronic Monitoring,Held Without Bond,Total
1-2,12.673,87.327,58.444,29.014,6.055,6.448,42.652
3-4,21.774,78.226,50.262,31.764,10.831,7.130,49.635
5-6,42.352,57.648,24.712,39.199,24.201,11.845,7.712
All Defendants,19.479,80.521,51.781,31.165,9.825,7.203,100.000


##### Pre-Reform

In [61]:
pre_reform_fta_hearing_outcomes.percentages()

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Electronic Monitoring,Held Without Bond,Total
1-2,20.651,79.349,35.378,44.314,18.627,0.903,45.003
3-4,32.909,67.091,19.771,50.460,28.425,0.949,48.741
5-6,49.272,50.728,11.366,62.878,24.356,1.064,6.256
All Defendants,28.416,71.584,26.269,48.471,23.761,0.935,100.000


### Release Outcomes

##### Post-Reform

In [62]:
post_reform_fta_release_outcomes = FTA_NCA_Release_Outcome_Table(post_reform_released, 
                                                                 ['FTAScale'])
post_reform_fta_release_outcomes

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
1-2,1506,1549,78,2606,11335
3-4,2210,2278,58,3710,11816
5-6,398,337,11,570,1353
All Defendants,4114,4164,147,6886,24504


##### Pre-Reform

In [63]:
pre_reform_fta_release_outcomes = FTA_NCA_Release_Outcome_Table(pre_reform_released, 
                                                                ['FTAScale'])
pre_reform_fta_release_outcomes

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
1-2,1459,1538,70,2569,10194
3-4,1865,1943,61,3095,9335
5-6,253,234,10,389,906
All Defendants,3577,3715,141,6053,20435


#### Percentages

##### Post-Reform

In [64]:
post_reform_fta_release_outcomes.percentages()

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
1-2,13.286,13.666,0.688,22.991,46.258
3-4,18.703,19.279,0.491,31.398,48.221
5-6,29.416,24.908,0.813,42.129,5.522
All Defendants,16.789,16.993,0.600,28.102,100.000


##### Pre-Reform

In [65]:
pre_reform_fta_release_outcomes.percentages()

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
1-2,14.312,15.087,0.687,25.201,49.885
3-4,19.979,20.814,0.653,33.155,45.681
5-6,27.925,25.828,1.104,42.936,4.434
All Defendants,17.504,18.180,0.690,29.621,100.000


## New Criminal Activity Subscale

This section analyzes hearing and release outcomes by NCA subscale scores. You'll notice a similar pattern here as with the FTA subscale above: After G.O. 18.8A, the percentage of defendants released pretrial increased signficantly, but NCA rates remained mostly the same, with perhaps a small increase. 

In the context of my article, the key finding is that **76 percent** of people who received a "high risk" score (i.e., five or six) on the NCA subscale didn't receive a new criminal charge during their release, compared with **83 percent** of defendants overall. 

### Hearing Outcomes

##### Post-Reform

In [66]:
post_reform_nca_hearing_outcomes = FTA_NCA_Hearing_Outcome_Table(post_reform, ['NCAScale'])
post_reform_nca_hearing_outcomes

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Average D-Bond (if set),Electronic Monitoring,Held Without Bond,Total
1-2,581,8523,5938,2353,33870,419,389,9104
3-4,3203,13107,8614,5078,32690,1433,1184,16310
5-6,2144,2874,1206,2053,44310,1138,619,5018
All Defendants,5928,24504,15758,9484,35500,2990,2192,30432


##### Pre-Reform

In [67]:
pre_reform_nca_hearing_outcomes = FTA_NCA_Hearing_Outcome_Table(pre_reform, ['NCAScale'])
pre_reform_nca_hearing_outcomes

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Average D-Bond (if set),Electronic Monitoring,Held Without Bond,Total
1-2,1008,7986,4056,3337,107850,1501,50,8994
3-4,5022,10370,3108,7785,99800,4238,170,15392
5-6,2082,2079,335,2715,105510,1044,47,4161
All Defendants,8112,20435,7499,13837,102860,6783,267,28547


#### Percentages

##### Post-Reform

In [68]:
post_reform_nca_hearing_outcomes.percentages()

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Electronic Monitoring,Held Without Bond,Total
1-2,6.382,93.618,65.224,25.846,4.602,4.273,29.916
3-4,19.638,80.362,52.814,31.134,8.786,7.259,53.595
5-6,42.726,57.274,24.033,40.913,22.678,12.336,16.489
All Defendants,19.479,80.521,51.781,31.165,9.825,7.203,100.000


##### Pre-Reform

In [69]:
pre_reform_nca_hearing_outcomes.percentages()

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Electronic Monitoring,Held Without Bond,Total
1-2,11.207,88.793,45.097,37.103,16.689,0.556,31.506
3-4,32.627,67.373,20.192,50.578,27.534,1.104,53.918
5-6,50.036,49.964,8.051,65.249,25.090,1.130,14.576
All Defendants,28.416,71.584,26.269,48.471,23.761,0.935,100.000


### Release Outcomes

##### Post-Reform

In [70]:
post_reform_nca_release_outcomes = FTA_NCA_Release_Outcome_Table(post_reform_released, ['NCAScale'])
post_reform_nca_release_outcomes

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
1-2,1086,930,54,1742,8523
3-4,2425,2546,73,4121,13107
5-6,603,688,20,1023,2874
All Defendants,4114,4164,147,6886,24504


##### Pre-Reform

In [71]:
pre_reform_nca_release_outcomes = FTA_NCA_Release_Outcome_Table(pre_reform_released, ['NCAScale'])
pre_reform_nca_release_outcomes

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
1-2,1100,1000,40,1814,7986
3-4,2014,2157,72,3408,10370
5-6,463,558,29,831,2079
All Defendants,3577,3715,141,6053,20435


#### Percentages

##### Post-Reform

In [72]:
post_reform_nca_release_outcomes.percentages()

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
1-2,12.742,10.912,0.634,20.439,34.782
3-4,18.502,19.425,0.557,31.441,53.489
5-6,20.981,23.939,0.696,35.595,11.729
All Defendants,16.789,16.993,0.600,28.102,100.000


##### Pre-Reform

In [73]:
pre_reform_nca_release_outcomes.percentages()

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
1-2,13.774,12.522,0.501,22.715,39.080
3-4,19.421,20.800,0.694,32.864,50.746
5-6,22.270,26.840,1.395,39.971,10.174
All Defendants,17.504,18.180,0.690,29.621,100.000


## New Violent Criminal Activity Subscale

This section analyzes hearing and release outcomes by NVCA flag. 

In the context of my article, the key finding is that **99.0 percent** of people who received an NVCA flag were not charged with a new violent crime after their release, compared with **99.4 percent** of defendants overall. 


### Hearing Outcomes

##### Post-Reform

In [74]:
post_reform_nvca_hearing_outcomes = NVCA_Hearing_Outcome_Table(post_reform, ['PSAVioFlag'])
post_reform_nvca_hearing_outcomes

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Average D-Bond (if set),Electronic Monitoring,Held Without Bond,Total
No NVCA Flag,4387,23466,15541,8388,30970,2696,1220,27853
NVCA Flag,1541,1038,217,1096,70220,294,972,2579
All Defendants,5928,24504,15758,9484,35500,2990,2192,30432


##### Pre-Reform

In [75]:
pre_reform_nvca_hearing_outcomes = NVCA_Hearing_Outcome_Table(pre_reform, ['PSAVioFlag'])
pre_reform_nvca_hearing_outcomes

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Average D-Bond (if set),Electronic Monitoring,Held Without Bond,Total
No NVCA Flag,6634,19688,7456,12064,84570,6512,139,26322
NVCA Flag,1478,747,43,1773,227240,271,128,2225
All Defendants,8112,20435,7499,13837,102860,6783,267,28547


#### Percentages

##### Post-Reform

In [76]:
post_reform_nvca_hearing_outcomes.percentages()

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Electronic Monitoring,Held Without Bond,Total
No NVCA Flag,15.751,84.249,55.797,30.115,9.679,4.380,91.525
NVCA Flag,59.752,40.248,8.414,42.497,11.400,37.689,8.475
All Defendants,19.479,80.521,51.781,31.165,9.825,7.203,100.000


##### Pre-Reform

In [77]:
pre_reform_nvca_hearing_outcomes.percentages()

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Electronic Monitoring,Held Without Bond,Total
No NVCA Flag,25.203,74.797,28.326,45.832,24.740,0.528,92.206
NVCA Flag,66.427,33.573,1.933,79.685,12.180,5.753,7.794
All Defendants,28.416,71.584,26.269,48.471,23.761,0.935,100.000


### Release Outcomes

##### Post-Reform

In [78]:
post_reform_nvca_release_outcomes = NVCA_Release_Outcome_Table(post_reform_released, ['PSAVioFlag'])
post_reform_nvca_release_outcomes

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
No NVCA Flag,3940,3965,137,6586,23466
NVCA Flag,174,199,10,300,1038
All Defendants,4114,4164,147,6886,24504


##### Pre-Reform

In [79]:
pre_reform_nvca_release_outcomes = NVCA_Release_Outcome_Table(pre_reform_released, ['PSAVioFlag'])
pre_reform_nvca_release_outcomes

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
No NVCA Flag,3443,3522,128,5796,19688
NVCA Flag,134,193,13,257,747
All Defendants,3577,3715,141,6053,20435


#### Percentages

##### Post-Reform

In [80]:
post_reform_nvca_release_outcomes.percentages()

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
No NVCA Flag,16.790,16.897,0.584,28.066,95.764
NVCA Flag,16.763,19.171,0.963,28.902,4.236
All Defendants,16.789,16.993,0.600,28.102,100.000


##### Pre-Reform

In [81]:
pre_reform_nvca_release_outcomes.percentages()

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
No NVCA Flag,17.488,17.889,0.65,29.439,96.345
NVCA Flag,17.938,25.837,1.74,34.404,3.655
All Defendants,17.504,18.180,0.69,29.621,100.000


You may notice that the NVCA rate for "high risk" defendants released before the reforms is nearly twice that of defendants released after the reforms. We can test the significance of this difference using a chi-squared test as follows:

In [82]:
# Find number of NVCAs and no NVCAs for defendants with NVCA flag released post-reform
post_reform_nvca_counts = post_reform.loc[
    post_reform['PSAVioFlag'] & ~(post_reform['NewVioCrimActivity'] == 2), 'NewVioCrimActivity'
].value_counts()


# Find number of NVCAs and no NVCAs for defendants with NVCA flag released pre-reform
pre_reform_nvca_counts = pre_reform.loc[
    pre_reform['PSAVioFlag'] & ~(pre_reform['NewVioCrimActivity'] == 2), 'NewVioCrimActivity'
].value_counts()

scs.chisquare(
    post_reform_nvca_counts,
    pre_reform_nvca_counts,
)

Power_divergenceResult(statistic=118.45252567595892, pvalue=1.3801830914497984e-27)

Note that the p-value is well below the typical standard of 0.05 -- in fact, it's about as close to zero as imaginable. This suggests that the difference between NVCA rates for "high risk" defendants in the pre- vs. post-G.O. 18.8A is statistically significant, which is interesting, because the post-reform NVCA rate for "high risk" defendants is *lower*, even though the release rate is higher. 

## PSA-DMF Outcomes

Like most jurisdictions that use the PSA, Cook County translates the PSA subscale scores into a release conditions recommendation using a "decision-making framework" (which Arnold Ventures has recently rebranded to "release conditions matrix"). The DMF plots FTA scores on one axis and NCA scores on the other, with a release recommendation defined for each possible pairing of FTA and NCA scores. (Note that not all combinations of FTA and NCA scores are possible, because the criteria used to calculate each score overlap. For instance, one can't get a 1 on the FTA scale and a 6 on the NCA scale, because meeting the criteria to get a 6 on the NCA scale would necessarily push you up to at least a 3 on the FTA scale.) 

Different jurisdictions handle the NVCA flag in various ways, but in Cook County, anyone with the NVCA flag is put in the highest risk category, regardless of their FTA and NCA scores. You can see Cook County's DMF here: LINKTK

You'll notice from the results below that combining FTA and NCA scores substantially reduces the PSA's accuracy. This is important, because it means the release recommendations don't necessarily correspond to increasing levels of risk. Throughout the article, I typically compare outcomes based on the subscale scores, but that's not how judges make their decisions. The DMF release recommendation is far more predictive of hearing outcomes than the FTA or NCA subscales, even though it's less predictive of defendants' behavior once released. 

One possible objection to this interpretation is that the DMF release recommendations also correspond to higher levels of pretrial supervision, i.e., a high risk defendant will be recommended for more stringent pretrial release conditions, which might be why they have lower-than-expected failure rates. It's an intuitive idea; someone on electronic monitoring might be less likely to reoffend or miss a court date than someone who is released without any conditions of supervision at all, even if the person on electronic monitoring is "riskier" according to the PSA. 

Unfortunately, the dataset released by Cook County does not make it easy to see how much of a factor different release conditions might have on defendants' behavior, because it doesn't include any data about what level of pretrial supervision each defendant actually received; it only tells us what level they were recommended to receive. (The one exception to this is electronic monitoring, which is recorded in the circuit court dataset. However, as you'll see below, defendants on electronic monitoring were not substantially less likely to reoffend than defendants on lower levels of supervision, even if you account for differences in average risk scores.)

A proxy for this is a court-watching dataset released by the Chicago Community Bond Fund early last year. It only includes data for August through October 2017, which somewhat limits its value, but it does have data on the level of supervision each defendant received in bond court. It doesn't tell us what happened when people were released, but it does give some idea of how well DMF recommendations corresponded to the actual conditions of supervision each person actually received. I may analyze this at some point in the future.

### Hearing Outcomes

##### Post-Reform

In [83]:
post_reform_dmf_hearing_outcomes = DMF_Hearing_Outcome_Table(post_reform, ['PSASuperRec'])
post_reform_dmf_hearing_outcomes

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Average D-Bond (if set),Electronic Monitoring,Held Without Bond,Total
Release No Conditions,109,5228,3843,1225,20990,224,43,5337
Pretrial Monitoring,461,5017,3531,1551,30380,295,100,5478
"If Released, Pretrial Superivision 1-3",1849,10147,6935,3638,27160,1010,411,11996
"If Released, PTS Plus Curfew",187,371,230,233,37460,70,25,558
"If Released, Court-Ordered Electronic Monitoring",657,1408,572,777,34640,620,95,2065
"If Released, Max Conditions",2665,2333,647,2060,62840,771,1518,4998
All Defendants,5928,24504,15758,9484,35500,2990,2192,30432


##### Pre-Reform

In [84]:
pre_reform_dmf_hearing_outcomes = DMF_Hearing_Outcome_Table(pre_reform, ['PSASuperRec'])
pre_reform_dmf_hearing_outcomes

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Average D-Bond (if set),Electronic Monitoring,Held Without Bond,Total
Release No Conditions,349,4991,2807,1705,72600,777,10,5340
Pretrial Monitoring,921,4397,1805,2177,74090,1289,8,5318
"If Released, Pretrial Superivision 1-3",3373,8034,2451,5473,71710,3383,48,11407
"If Released, PTS Plus Curfew",216,278,46,322,74700,117,2,494
"If Released, Court-Ordered Electronic Monitoring",734,1040,199,993,75570,570,7,1774
"If Released, Max Conditions",2519,1695,191,3167,204250,647,192,4214
All Defendants,8112,20435,7499,13837,102860,6783,267,28547


#### Percentages

##### Post-Reform

In [85]:
post_reform_dmf_hearing_outcomes.percentages()

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Electronic Monitoring,Held Without Bond,Total
Release No Conditions,2.042,97.958,72.007,22.953,4.197,0.806,17.537
Pretrial Monitoring,8.415,91.585,64.458,28.313,5.385,1.825,18.001
"If Released, Pretrial Superivision 1-3",15.413,84.587,57.811,30.327,8.419,3.426,39.419
"If Released, PTS Plus Curfew",33.513,66.487,41.219,41.756,12.545,4.480,1.834
"If Released, Court-Ordered Electronic Monitoring",31.816,68.184,27.700,37.627,30.024,4.600,6.786
"If Released, Max Conditions",53.321,46.679,12.945,41.216,15.426,30.372,16.424
All Defendants,19.479,80.521,51.781,31.165,9.825,7.203,100.000


##### Pre-Reform

In [86]:
pre_reform_dmf_hearing_outcomes.percentages()

,Incarcerated Pretrial,Released Pretrial,I-Bonds,D-Bonds,Electronic Monitoring,Held Without Bond,Total
Release No Conditions,6.536,93.464,52.566,31.929,14.551,0.187,18.706
Pretrial Monitoring,17.319,82.681,33.941,40.936,24.238,0.150,18.629
"If Released, Pretrial Superivision 1-3",29.570,70.430,21.487,47.979,29.657,0.421,39.959
"If Released, PTS Plus Curfew",43.725,56.275,9.312,65.182,23.684,0.405,1.730
"If Released, Court-Ordered Electronic Monitoring",41.375,58.625,11.218,55.975,32.131,0.395,6.214
"If Released, Max Conditions",59.777,40.223,4.533,75.154,15.354,4.556,14.762
All Defendants,28.416,71.584,26.269,48.471,23.761,0.935,100.000


### Release Outcomes

##### Post-Reform

In [87]:
post_reform_dmf_release_outcomes = DMF_Release_Outcome_Table(post_reform_released, ['PSASuperRec'])
post_reform_dmf_release_outcomes

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
Release No Conditions,653,574,35,1060,5228
Pretrial Monitoring,704,662,18,1186,5017
"If Released, Pretrial Superivision 1-3",1951,2037,64,3294,10147
"If Released, PTS Plus Curfew",59,95,2,127,371
"If Released, Court-Ordered Electronic Monitoring",279,303,5,458,1408
"If Released, Max Conditions",468,493,23,761,2333
All Defendants,4114,4164,147,6886,24504


##### Pre-Reform

In [88]:
pre_reform_dmf_release_outcomes = DMF_Release_Outcome_Table(pre_reform_released, ['PSASuperRec'])
pre_reform_dmf_release_outcomes

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
Release No Conditions,649,666,27,1140,4991
Pretrial Monitoring,694,588,17,1088,4397
"If Released, Pretrial Superivision 1-3",1603,1714,52,2707,8034
"If Released, PTS Plus Curfew",47,76,5,103,278
"If Released, Court-Ordered Electronic Monitoring",262,255,13,416,1040
"If Released, Max Conditions",322,416,27,599,1695
All Defendants,3577,3715,141,6053,20435


#### Percentages

##### Post-Reform

In [89]:
post_reform_dmf_release_outcomes.percentages()

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
Release No Conditions,12.490,10.979,0.669,20.275,21.335
Pretrial Monitoring,14.032,13.195,0.359,23.640,20.474
"If Released, Pretrial Superivision 1-3",19.227,20.075,0.631,32.463,41.410
"If Released, PTS Plus Curfew",15.903,25.606,0.539,34.232,1.514
"If Released, Court-Ordered Electronic Monitoring",19.815,21.520,0.355,32.528,5.746
"If Released, Max Conditions",20.060,21.132,0.986,32.619,9.521
All Defendants,16.789,16.993,0.600,28.102,100.000


##### Pre-Reform

In [90]:
pre_reform_dmf_release_outcomes.percentages()

,"Released, FTAs","Released, NCAs","Released, NVCAs","Released, Any Failure",Total
Release No Conditions,13.003,13.344,0.541,22.841,24.424
Pretrial Monitoring,15.783,13.373,0.387,24.744,21.517
"If Released, Pretrial Superivision 1-3",19.953,21.334,0.647,33.694,39.315
"If Released, PTS Plus Curfew",16.906,27.338,1.799,37.050,1.360
"If Released, Court-Ordered Electronic Monitoring",25.192,24.519,1.250,40.000,5.089
"If Released, Max Conditions",18.997,24.543,1.593,35.339,8.295
All Defendants,17.504,18.180,0.690,29.621,100.000


# Gun Analysis

Backup for the gun possession stats I cite in the piece:

In [91]:
# Filter post-reform data to only include those with weapons charges who were released pretrial
weapons_released = post_reform.loc[(post_reform['ChargeCategory'] == 2) & post_reform['PretrialRelease']]

# The number of people with weapons charges released since Sept. 2017
len(weapons_released)

3288

In [92]:
# Find the number of people released who later received an NVCA
weap_NVCA = len(weapons_released.loc[weapons_released['NewVioCrimActivity'] == 1])
weap_NVCA

27

In [93]:
# Calculate the percentage of people released who later received an NVCA
percent_released_NVCA = weap_NVCA / len(weapons_released)
print(f"{100*percent_released_NVCA:.3} percent received an NVCA")

0.821 percent received an NVCA


# Drug Possession Analysis

Backup for my claim that most people with drug charges are typically released.

In [94]:
# Filter pre-reform data to only include those with drug charges
drugs = pre_reform.loc[pre_reform['ChargeCategory'] == 0]

# Calculate the number who were released
drugs_released = drugs.loc[drugs['PretrialRelease']]

# Calculate the percentage of people with drug charges who were released pretrial
percent_released = len(drugs_released) / len(drugs)
print(f'{100*percent_released:.3} percent were released')

79.9 percent were released


In [95]:
drug_bond = drugs.loc[drugs['InitialBondOrder'] == 1]
drug_high_bond = drug_bond.loc[drug_bond['DBond10pAmt'] >= 5000]
drug_high_bond['PretrialRelease'].value_counts()

False    713
True     511
Name: PretrialRelease, dtype: int64